# Question 4

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/DriverTraining-ForInClassLearning-2024.csv')
df
#print(df.to_string())

# a)

## 1.

In [ ]:
print("Percent of data missing:")
100*(df.isnull().sum())/len(df)

## 2.

Most of the issues are with missing data, but some area codes are extended, and some cities have different capitalization or punctuation.
To solve the inconsistencies with the cities we can make all cities lower case and remove punctuation.
For extended zip codes we can trim anything coming after a dash.
For missing values, the row can be removed if we are looking at that value specifically.

One minor issue is that the dates have two digit years which doesnt really play well with the library I'm using so I fixed that below.

My data cleaning strategy will mostly involve ignoring bad data.

In [ ]:
from datetime import datetime
# 4 digit years
def formatDate(date):
    if (not isinstance(date, str)):
        return
    date_split = date.split('/')
    # Y2K 0_0
    prefix = '20'
    # Should check current year instead of hard coding 24
    if (int(date_split[2]) > 24):
        prefix = '19'
    return date[:len(date) - 2] + prefix + date[len(date) - 2:]

# Applying formatting to all rows
for _, row in df.iterrows():
    row['class_date'] = formatDate(row['class_date'])        
    row['dob'] = formatDate(row['dob']) 
df

# b)

## 1.

In [ ]:
from dateutil.parser import parse

# Ignore all null genders
mfDf = df[~df['gender'].isnull()]
mPercent = 100*len(mfDf[mfDf['gender'] == 'Male'])/len(mfDf)
fPercent = 100*len(mfDf[mfDf['gender'] == 'Female'])/len(mfDf)
print('All Data:')
print('Percent male:   ', mPercent)
print('Percent female: ', fPercent)
# Getting rid of pandas warning
pd.options.mode.chained_assignment = None
mfDf["class_date"] = mfDf["class_date"].apply(lambda x: parse(x))
# Group by month
g = mfDf.groupby(pd.Grouper(key='class_date', freq='ME'))
dfs = [group for _, group in g]
# Don't feel like doing this the right way
month = 1
dfsByMonth = [0] * 12
for frame in dfs:
    if (month == 12):
        month = 0
    # dfsByMonth hasn't seen this month yet
    if (isinstance(dfsByMonth[month], int)):
        dfsByMonth[month] = frame
    else:
        dfsByMonth[month] = pd.concat([dfsByMonth[month], frame])
    month += 1

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
for idx, monthDf in enumerate(dfsByMonth):
    mPerc = 100*len(monthDf[monthDf['gender'] == 'Male'])/len(monthDf)
    fPerc = 100*len(monthDf[monthDf['gender'] == 'Female'])/len(monthDf)
    print('')
    print(months[idx], ':')
    print('Percent male:   ', mPerc)
    print('Percent female: ', fPerc)

I don't see an overall pattern, but there seem to be much more males in Oct and Dec

## 2.

In [ ]:
# Ignore all null DOB
mfDf = mfDf[~mfDf['dob'].isnull()]
mfDf["dob"] = mfDf["dob"].apply(lambda x: parse(x))
mfDf["age-days"] = mfDf["class_date"] - mfDf["dob"]
totalAvg = mfDf["age-days"].mean()
genderedMeans = mfDf.groupby('gender')["age-days"].mean()
print("Total   ", totalAvg)
print(genderedMeans)

One major assumption I made is that missing dob isn't due to the participant being younger or older than the average. I'm assuming the missing DOBs are uniformly random. I also chose to use days rather than years based on the assumption that all of the participants would be around 16 years old and years wouldn't show the difference.

## 3.

In [ ]:
# Drop all rows without city data (Decided to just make them 'not given')
# cityDf = df[df['city'].notna()]
def formatCity(city):
    newCity = city.lower()
    newCity = ''.join(filter(str.isalnum, newCity))
    # Mount Pleasant has a very high amount of misspelling
    mtPleasant = {'mtpleasent', 'mtpleasant', 'mountpleasent'}
    if (newCity in mtPleasant):
        newCity = "mountpleasant"
    return newCity

# Didnt account for different states, but most of this is from SC
for _, row in df.iterrows():
    if (not isinstance(row['city'], str)):
        row['city'] = 'not given'
    else:
        row['city'] = formatCity(row['city'])

# A bunch of random spelling so I'm just dropping anything under 0.04%
# This removes places like John's Island, but since the majority are missplelligns im okay with it
counts = df['city'].value_counts(normalize=True).mul(100)
idx = 0
for _, val in counts.items():
    if (val <= 0.04):
        break
    idx += 1
counts = counts.iloc[:idx]
counts.plot(kind = 'bar', ylabel='Percent of total', figsize=(18,5))
# Idealy I would plug in the city names to some fuzzy search algorithm to match them with a list of cities and towns